Esquemas de partición de los datos
===

* *15 min* | Ultima modificación: Junio 22, 2019

Los resultados de un modelo son dependientes de la forma en que se partan los datos. En este tutorial se presentan diferentes esquemas para la partición de los datos.

## Conjuntos de calibración, prueba y predicción

Los resultados de la evaluación de los modelos es dependiente de los datos usados. En la práctica, los datos se suelen partir en tres conjuntos, tal como muestra la gráfica de abajo:


* Conjunto de calibración de parámetros del modelo.


* Conjunto de prueba, usado comunmente para determinar la complejidad del modelo o el valor óptimo de alguno de sus parámetros.


* Conjunto de pronóstico, en el que se intenta reproducir el comportamiento del modelo en productivo.

![alt text](assets/data-partition.jpg)

En la figura anterior, los datos se dividen secuencialmente, pero podría construirse cada conjunto aletaoriamente. 

Si se tiene en cuenta que hay muchas particiones aleatorias posibles, una mejor estimación de la matriz de confusión (o cualquier otro estadístico que se calcule para un conjunto de datos) podría ser tomando los valores esperados de cada métrica. Es decir, si se repite el experimento $N$ veces, se tendrían $N$ valores posibles para cada uno de los elementos de la matriz de confusión y por lo tanto se podría tener su valor medio. Esta sería una métrica mucho más apropiada.

## Esquemas de partición de los datos

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

%matplotlib inline
%load_ext rpy2.ipython

### Muestra de datos

In [2]:
#
# Se usaran dos listas para representar los
# indices de los patrones de la muestra de datos
#
X, y = list(range(20)), list(range(20))
X

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [3]:
y

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

### train_test_split

In [4]:
#
# Permite dividir la muestra en tres conjuntos de datos
#
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,  ## datos originales
    test_size=2,  ## float/int, tamaño de la muestra de prueba
    random_state=42,
)  ## semilla del generador aleatorio

#
# Muestra de entrenamiento
#
X_train

[15, 1, 8, 5, 11, 3, 18, 16, 13, 2, 9, 19, 4, 12, 7, 10, 14, 6]

In [5]:
#
# Datos de entrenamiento para la variable respuesta
#
y_train

[15, 1, 8, 5, 11, 3, 18, 16, 13, 2, 9, 19, 4, 12, 7, 10, 14, 6]

In [6]:
#
# Muestra de prueba
#
X_test

[0, 17]

In [7]:
#
# Datos de prueba para la variable respuesta
#
y_test

[0, 17]

### K-fold crossvalidation

En este método, el conjunto de datos para entrenamiento (ajuste + prueba) es dividido en $K$ grupos. Este es un proceso iterativo que opera de la siguiente forma (véase la figura de abajo). 


* Se toma el grupo 1 como conjunto de datos de prueba (grupo rojo) y se entrena el modelo con los grupos restantes {2, ..., K} (grupo negro).


* Se toma el grupo 2 como conjunto de datos de prueba (grupo rojo) y se entrena el modelo con los grupos restantes {1, 3, ..., K} (grupo negro).


* Se continua de esta forma hasta que se usa el grupo K para prueba, mientras que se usan los grupos 1 hasta K-1 para entrenamiento.



![assets/k-fold-crossval.jpg](assets/k-fold-crossval.jpg)

De esta forma, se tienen K valores posibles para el estadístico de interés. Usualmente se reporta su valor promedio.

Note que una mejor opción sería distribuir los datos en cada grupo de forma aleatoria.

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)  ## se generan tres grupos
for train, test in kf.split(X):
    print("%s %s" % (train, test))

[ 7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 2 3 4 5 6]
[ 0  1  2  3  4  5  6 14 15 16 17 18 19] [ 7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] [14 15 16 17 18 19]


### Repeated KFold

In [9]:
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=4, n_repeats=2, random_state=123)

for train, test in rkf.split(X):
    print("%s %s" % (train, test))

[ 0  1  2  3  6  7  9 10 11 12 13 15 16 18 19] [ 4  5  8 14 17]
[ 1  2  3  4  5  6  8  9 10 11 13 14 16 17 18] [ 0  7 12 15 19]
[ 0  1  2  4  5  6  7  8 12 13 14 15 17 18 19] [ 3  9 10 11 16]
[ 0  3  4  5  7  8  9 10 11 12 14 15 16 17 19] [ 1  2  6 13 18]
[ 0  1  2  3  4  7  9 10 12 14 15 16 17 18 19] [ 5  6  8 11 13]
[ 0  1  2  3  4  5  6  7  8 11 13 14 16 17 19] [ 9 10 12 15 18]
[ 0  4  5  6  8  9 10 11 12 13 14 15 16 18 19] [ 1  2  3  7 17]
[ 1  2  3  5  6  7  8  9 10 11 12 13 15 17 18] [ 0  4 14 16 19]


### Leave-One-Out 

Este es el K-fold con K=1.

![assets/leave-one-out.jpg](assets/leave-one-out.jpg)

In [10]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()

for train, test in loo.split(X):
    print("%s %s" % (train, test))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0]
[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [1]
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [2]
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [3]
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [4]
[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [5]
[ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 15 16 17 18 19] [6]
[ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19] [7]
[ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19] [8]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19] [9]
[ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19] [10]
[ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19] [11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19] [12]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19] [13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 15 16 17 18 19] [14]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 1

### Leave-P-Out

In [11]:
from sklearn.model_selection import LeavePOut

lpo = LeavePOut(p=3)

n = 0
for train, test in lpo.split(X):
    print("%s %s" % (train, test))
    n += 1
    if n > 10:
        break

[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 2]
[ 2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 3]
[ 2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 4]
[ 2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 5]
[ 2  3  4  5  7  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 6]
[ 2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19] [0 1 7]
[ 2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19] [0 1 8]
[ 2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19] [0 1 9]
[ 2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19] [ 0  1 10]
[ 2  3  4  5  6  7  8  9 10 12 13 14 15 16 17 18 19] [ 0  1 11]
[ 2  3  4  5  6  7  8  9 10 11 13 14 15 16 17 18 19] [ 0  1 12]


### ShuffleSplit

In [12]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(
    n_splits=3,  ## número de particiones
    test_size=0.25,  ## porcentaje de prueba
    random_state=0,
)  ## semilla aleatoria 0

for train_index, test_index in ss.split(X):
    print("%s %s" % (train_index, test_index))

[17  6 13  4  2  5 14  9  7 16 11  3  0 15 12] [18  1 19  8 10]
[12 19 16 10  0  3  4 15  8 13  9  5 14  7  6] [11  1 18 17  2]
[ 2  8  6  3 17  4 10 16 18  9  1  0  7 14 19] [15 13 12  5 11]


### Stratifed k-fold

Se usa en problemas de clasificación en los que la distribución porcentual de las clases en los grupos de entrenamiento y prueba son similares a los de la muestra original.

In [13]:
from sklearn.model_selection import StratifiedKFold

Xk = np.ones(10)
yk = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

skf = StratifiedKFold(n_splits=3)  ## número de particiones

for train, test in skf.split(Xk, yk):
    print("%s %s" % (train, test))

[2 3 6 7 8 9] [0 1 4 5]
[0 1 3 4 5 8 9] [2 6 7]
[0 1 2 4 5 6 7] [3 8 9]


### Repeated KFold

In [14]:
from sklearn.model_selection import RepeatedKFold

#
# Se repite K-Fold n veces
#
rkf = RepeatedKFold(n_splits=4, n_repeats=2, random_state=123)

for train, test in rkf.split(X):
    print("%s %s" % (train, test))

[ 0  1  2  3  6  7  9 10 11 12 13 15 16 18 19] [ 4  5  8 14 17]
[ 1  2  3  4  5  6  8  9 10 11 13 14 16 17 18] [ 0  7 12 15 19]
[ 0  1  2  4  5  6  7  8 12 13 14 15 17 18 19] [ 3  9 10 11 16]
[ 0  3  4  5  7  8  9 10 11 12 14 15 16 17 19] [ 1  2  6 13 18]
[ 0  1  2  3  4  7  9 10 12 14 15 16 17 18 19] [ 5  6  8 11 13]
[ 0  1  2  3  4  5  6  7  8 11 13 14 16 17 19] [ 9 10 12 15 18]
[ 0  4  5  6  8  9 10 11 12 13 14 15 16 18 19] [ 1  2  3  7 17]
[ 1  2  3  5  6  7  8  9 10 11 12 13 15 17 18] [ 0  4 14 16 19]
